In [1]:
import json
import pandas as pd

In [2]:
with open('notas.json', 'r', encoding='utf-8') as fp:
    notas = json.load(fp)

df = pd.json_normalize(notas, record_path=['retorno', 'disciplinas'])
# print(notas['retorno'])
print(df)


   disciplina turma  nota                                   situacao
0       EM919   A     9.5  Aprovado por Nota / Conceito e Frequência
1       EM914   A    10.0  Aprovado por Nota / Conceito e Frequência
2       EM916   A      --                    Aprovado por Frequência
3       EM918   A      --                                Desistência
4       EM926   A     7.6  Aprovado por Nota / Conceito e Frequência
..        ...   ...   ...                                        ...
71      LA122          --                              Proeficiência
72      LA222          --                              Proeficiência
73      MA111   3     5.8  Aprovado por Nota / Conceito e Frequência
74      MA141   D     5.6  Aprovado por Nota / Conceito e Frequência
75      QG100   I     6.9  Aprovado por Nota / Conceito e Frequência

[76 rows x 4 columns]


In [2]:
from pypdf import PdfReader

reader = PdfReader("HistoricoEscolarAluno (1).pdf")

for page in reader.pages:
    print(page.extract_text())

UNIVERSIDADE ESTADUAL DE CAMPINAS
Histórico EscolarDiretoria Acadêmica
 
 
 Documento de Identidade  Nascimento  Sexo
 Naturalidade  Nacionalidade27/09/1996 41538957-4-SP Masculino
São Paulo - SP Brasileira CPF
350.620.468-82157462 Registro Acadêmico
Victor Colella Gomes Castro Pereira Nome
 Dados do Ingresso  Nível
Graduação Curso: 10 -  Engenharia Mecânica
Reconhecido pelo Decreto Federal n°  70733 de 19/06/1972 e renovado pela
Portaria CEE/GP n°  451 de 05/12/2018
 Forma de Ingresso Mês/Ano de Ingresso
Vestibular COMVEST 02/2014
Redação 7,50 Questões 8,13Notas do Vestibular (FASE: 1)
Português 5,10 Matemática 7,71
Inglês 7,29 Ciências Humanas e Artes 5,56
Ciências da Natureza 8,18Notas do Vestibular (FASE: 2)
 Escola Anterior
Colégio Anglo Leonardo da Vinci Mês/Ano da Conclusão
12/2013
 Situação no Curso  Ano de Catálogo  Ano da Turma
Concluiu em 07/12/2019
Colou grau em 06/03/20202014 2014
 Índices de Desempenho do aluno no Curso
Coeficiente de Rendimento Coeficiente de Rendimento 

In [2]:
import pdfplumber
import re
import pandas as pd
import numpy as np


def extract_table_from_pdf(pdf_path, page_number=0):
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables.append(page.extract_tables())  # Assuming there is only one table on the page
        return tables
# Replace 'your_pdf_file.pdf' with the path to your PDF file
pdf_path = "HistoricoEscolarAluno (1).pdf"
table_data = extract_table_from_pdf(pdf_path)

# Print the extracted table data
# for row in table_data:
#     print(row)

In [54]:
df = pd.DataFrame(columns=['code', 'name', 'grade', 'credits_hour', 'credits', 'other'])
pattern = r"(?P<code>[A-Z]{1,2}\s{0,1}\d{1,3})\s+(?P<name>.+?)\s+(?P<grade>\d+,\d+)\s+(?P<ch>\d+)\s+(?P<crd>\d+)\s+(?P<other_info>.+)"

for i in table_data:
    for j in i:
        for k in j:
            # Access the captured groups
            if k is not None:
                match = re.match(pattern, ''.join([item for item in k if item]))
                # print(''.join([item for item in k if item]))
                if match and 'Aprovado' in match.group('other_info'):
                    course_code = match.group('code')
                    course_name = match.group('name')
                    grade = match.group('grade')
                    ch = match.group('ch')
                    crd = match.group('crd')
                    other_info = match.group('other_info').strip()
                    df.loc[len(df)] = [course_code, course_name, float(grade.replace(',','.')), int(ch), int(crd), other_info]

In [60]:
print(f"Final GPA : {np.sum(df['credits'] * df['grade']) / df['credits'].sum():.2f}")

Final GPA : 6.98
